In [1]:
pip install numpy==1.25.2

   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.5 MB 487.6 kB/s eta 0:00:32
   ---------------------------------------- 0.1/15.5 MB 744.7 kB/s eta 0:00:21
   ---------------------------------------- 0.1/15.5 MB 847.9 kB/s eta 0:00:19
    --------------------------------------- 0.3/15.5 MB 1.2 MB/s eta 0:00:13
    --------------------------------------- 0.3/15.5 MB 1.2 MB/s eta 0:00:13
   - -------------------------------------- 0.5/15.5 MB 1.5 MB/s eta 0:00:10
   - -------------------------------------- 0.7/15.5 MB 1.8 MB/s eta 0:00:09
   -- ------------------------------------- 0.8/15.5 MB 1.9 MB/s eta 0:00:08
   -- ------------------------------------- 1.0/15.5 MB 2.2 MB/s eta 0:00:07
   --- ------------------------------------ 1.2/15.5 MB 2.3 MB/s eta 0:00:07
   --- -------

In [2]:
pip install sentence_transformers~=2.2.2

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install mteb

In [16]:
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
import numpy as np
import mteb

ModuleNotFoundError: No module named 'mteb'

In [4]:
pip freeze

asttokens==2.4.1
certifi==2024.6.2
charset-normalizer==3.3.2
click==8.1.7
colorama==0.4.6
comm==0.2.1
debugpy==1.8.0
decorator==5.1.1
executing==2.0.1
filelock==3.15.3
fsspec==2024.6.0
huggingface-hub==0.23.4
idna==3.7
intel-openmp==2021.4.0
ipykernel==6.29.0
ipython==8.20.0
jedi==0.19.1
Jinja2==3.1.4
joblib==1.4.2
jupyter_client==8.6.0
jupyter_core==5.7.1
MarkupSafe==2.1.5
matplotlib-inline==0.1.6
mkl==2021.4.0
mpmath==1.3.0
nest-asyncio==1.6.0
networkx==3.3
nltk==3.8.1
numpy==1.25.2
packaging==23.2
parso==0.8.3
pillow==10.3.0
platformdirs==4.1.0
prompt-toolkit==3.0.43
psutil==5.9.8
pure-eval==0.2.2
Pygments==2.17.2
python-dateutil==2.8.2
pywin32==306
PyYAML==6.0.1
pyzmq==25.1.2
regex==2024.5.15
requests==2.32.3
safetensors==0.4.3
scikit-learn==1.5.0
scipy==1.13.1
sentence-transformers==2.2.2
sentencepiece==0.2.0
six==1.16.0
stack-data==0.6.3
sympy==1.12.1
tbb==2021.13.0
threadpoolctl==3.5.0
tokenizers==0.19.1
torch==2.3.1
torchvision==0.18.1
tornado==6.4
tqdm==4.66.4
traitlets==5.14.

In [5]:
#defining functions:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [6]:
# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = ['query: how much protein should a female eat',
               'query: 南瓜的家常做法',
               "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
               "passage: 1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"]

In [7]:
#load models:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-small')
model_with_e5 = AutoModel.from_pretrained('intfloat/multilingual-e5-small')
model_with_sentence_transformer = SentenceTransformer('intfloat/multilingual-e5-small')

In [8]:
# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
batch_dict

{'input_ids': tensor([[     0,     41,   1294,     12,   3642,   5045,  21308,   5608,     10,
         117776,  73203,      2,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              

In [9]:
outputs = model_with_e5(**batch_dict)
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.2526, -0.0612, -0.2972,  ...,  0.1744,  0.0097,  0.1362],
         [ 0.1764, -0.3090, -0.2646,  ...,  0.1445,  0.1458,  0.2989],
         [ 0.1566, -0.3100, -0.2594,  ...,  0.1459,  0.1691,  0.2915],
         ...,
         [ 0.3977, -0.0538, -0.2153,  ...,  0.1540,  0.0514,  0.1949],
         [ 0.3768, -0.0546, -0.2243,  ...,  0.1711,  0.0454,  0.1951],
         [ 0.3611, -0.0538, -0.2445,  ...,  0.1663,  0.0781,  0.1949]],

        [[ 0.2138, -0.0392, -0.2063,  ...,  0.3219,  0.1744, -0.0456],
         [ 0.3034, -0.0494, -0.2561,  ...,  0.3990,  0.3571,  0.0787],
         [ 0.3038, -0.0386, -0.2365,  ...,  0.3948,  0.3628,  0.0766],
         ...,
         [ 0.3162,  0.0336, -0.1920,  ...,  0.4149,  0.3290,  0.0169],
         [ 0.2990,  0.0231, -0.2115,  ...,  0.4166,  0.3110,  0.0275],
         [ 0.2938,  0.0137, -0.2153,  ...,  0.4144,  0.3266,  0.0344]],

        [[ 0.1229, -0.0880, -0.1926,  ...,  0.3122,  

In [10]:
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
embeddings

tensor([[ 0.2113, -0.1942, -0.2871,  ...,  0.1289,  0.1541,  0.2488],
        [ 0.2518, -0.0720, -0.1733,  ...,  0.3655,  0.2942,  0.0529],
        [-0.1302, -0.2040, -0.2224,  ...,  0.3290,  0.2358,  0.2935],
        [ 0.0924, -0.0120, -0.1785,  ...,  0.2917,  0.3686,  0.0910]],
       grad_fn=<DivBackward0>)

In [11]:
# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
embeddings

tensor([[ 0.0422, -0.0388, -0.0573,  ...,  0.0257,  0.0308,  0.0497],
        [ 0.0534, -0.0153, -0.0367,  ...,  0.0775,  0.0623,  0.0112],
        [-0.0258, -0.0404, -0.0440,  ...,  0.0651,  0.0467,  0.0581],
        [ 0.0194, -0.0025, -0.0375,  ...,  0.0613,  0.0775,  0.0191]],
       grad_fn=<DivBackward0>)

In [12]:
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())

[[92.72809600830078, 75.96249389648438], [74.38050842285156, 90.60895538330078]]


In [13]:
embeddings_with_sentence_transformer = model_with_sentence_transformer.encode(input_texts, normalize_embeddings=True)
scores_with_sentence_transformer = (embeddings_with_sentence_transformer[:2] @ embeddings_with_sentence_transformer[2:].T) * 100
print(scores_with_sentence_transformer.tolist())

[[92.72811126708984, 75.96250915527344], [74.3805160522461, 90.60896301269531]]


In [14]:
embeddings_with_sentence_transformer

array([[ 0.0421759 , -0.03877136, -0.05730906, ...,  0.025732  ,
         0.03076708,  0.04966562],
       [ 0.05337086, -0.01525041, -0.03673498, ...,  0.07745242,
         0.06234608,  0.01122016],
       [-0.02577215, -0.04037194, -0.04400872, ...,  0.06510442,
         0.04665779,  0.05807815],
       [ 0.01940993, -0.00252936, -0.03752143, ...,  0.06130136,
         0.07746346,  0.01912975]], dtype=float32)

some improvement observed, although negligible 